### Ноутбук с рецептами работы в Спарк

In [1]:
import os
import sys

In [2]:
import pandas as pd
from scipy import stats

from importlib import reload
import time
from tqdm import tqdm

In [3]:
print(sys.version)

3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:17) [GCC 12.2.0]


### Загрузка пакетов и данных

In [4]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer
import pandserv as pds # сервисные функции для пандас и не только

Создание сессии Спарк с подключением к кластеру упаковано в класс SparkApp.  
Обращение с полями датафрейма Спарк- создание псевдонимов - в классе Cols

In [5]:
from sparkserv import SparkApp, Cols

In [6]:
import pyspark
import pyspark.sql.functions as f

In [7]:
print(pyspark.__version__)

3.4.1


In [8]:
spark_app = SparkApp(my_logger_create_level = 'INFO')

INFO:spark_app:spark_master_ip: 172.18.0.2
INFO:spark_app:pyspark version: 3.4.1
INFO:spark_app:starting building spark app object: pyspark-taxi-forecasting
INFO:spark_app:Spark app object built as: <pyspark.sql.session.SparkSession object at 0x7f23da56db50>
INFO:spark_app:==================================================================
INFO:spark_app:Spark object can be accessed as the SparkApp_object.spark property
INFO:spark_app:==================================================================


In [9]:
spark_master_ip = spark_app.get_spark_master_ip()
# print(spark_master_ip)

INFO:spark_app:spark_master_ip: 172.18.0.2


In [10]:
# spark = spark_app.build_spark_app(spark_master_ip=spark_master_ip)
spark = spark_app.spark

In [11]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 55.7 ms, sys: 23 ms, total: 78.7 ms
Wall time: 1min 33s


In [12]:
taxi.take(2)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12', Taxi ID='449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='

Интерфейс ДатаФрейма работает

In [13]:
taxi_rdd = spark.sparkContext.textFile('/work/data/Taxi_Trips_-_2022.csv')

In [14]:
taxi_rdd.take(5)

['Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid  Location',
 'bcfa19f2539021c054809d4c3993d226996ae095,368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334,01/01/2022 12:00:00 AM,01/01/2022 12:00:00 AM,152,0.1,,,,,3.75,0.00,0.00,0.00,3.75,Cash,Medallion Leasin,,,,,,',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12,449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317,01/01/2022 12:00:00 AM,01/01/2022 12:30:00 AM,2360,17.44,,,,8,47.75,0.00,0.00,5.00,52.75,Cash,Flash Cab,,,,41.899602111,-87.633308037,POINT (-87.6333080367 41.899

Интерфейс РДД также работает

In [15]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

### Класс Cols- псевдонимы имен полей ДФ

In [16]:
ct = Cols(taxi)

ct.get_aliases()

{'trip_i': 'Trip ID',
 'taxi_i': 'Taxi ID',
 'trip_st': 'Trip Start Timestamp',
 'trip_et': 'Trip End Timestamp',
 'trip_s': 'Trip Seconds',
 'trip_m': 'Trip Miles',
 'pickup_ct': 'Pickup Census Tract',
 'dropoff_ct': 'Dropoff Census Tract',
 'pickup_ca': 'Pickup Community Area',
 'dropoff_ca': 'Dropoff Community Area',
 'fare': 'Fare',
 'tips': 'Tips',
 'tolls': 'Tolls',
 'extras': 'Extras',
 'trip_t': 'Trip Total',
 'payment_t': 'Payment Type',
 'company': 'Company',
 'pickup_cl': 'Pickup Centroid Latitude',
 'pickup_clon': 'Pickup Centroid Longitude',
 'pickup_cloc': 'Pickup Centroid Location',
 'dropoff_cl': 'Dropoff Centroid Latitude',
 'dropoff_clon': 'Dropoff Centroid Longitude',
 'dropoff_cloc': 'Dropoff Centroid  Location'}

In [17]:
# Вывод значения - полного имени поля для псевдонима
ct.trip_i

'Trip ID'

### Пандас- показать полное содержимое ячеек столбцов

In [18]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [19]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';',index_col='Column Name')

table_descr

,Description,Type,Описание,Тип,Unnamed: 5
Column Name,,,,,
Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст,Ошибка:510
Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст,NaN
Trip Start Timestamp,When the trip started rounded to the nearest 15 minutes.,Date & Time,Время начала поездки округленное до ближайших 15 минут,Дата и время,NaN
Trip End Timestamp,When the trip ended rounded to the nearest 15 minutes.,Date & Time,Время окончания поездки округленное до ближайших 15 минут,Дата и время,NaN
Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число,NaN
Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число,NaN
Pickup Census Tract,The Census Tract where the trip began. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где началась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст,NaN
Dropoff Census Tract,The Census Tract where the trip ended. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где завершилась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст,NaN
Pickup Community Area,The Community Area where the trip began. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где началась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число,NaN


In [20]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [21]:
# %%time
# describe_result = taxi.describe().toPandas()

# time cost i5 (7g), 4 workers (x) 2gb RAM 
# CPU times: user 352 ms, sys: 32.2 ms, total: 384 ms
# Wall time: 2min 7s

In [22]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [23]:
dr = pd.read_csv('/work/data/2022_describe.csv',sep=',',index_col='summary')

In [24]:
drf = dr.copy()
pds.get_df_formated(drf, '`',2, 10)

In [25]:
drf.transpose()

summary,count,mean,stddev,min,max
Trip ID,6`382`425,nan,nan,000000bb18,ffffff1aae
Taxi ID,6`382`425,nan,nan,0041f8f0c9,fff84aa08a
Trip Start Timestamp,6`382`425,nan,nan,01/01/2022,12/31/2022
Trip End Timestamp,6`382`213,nan,nan,01/01/2022,12/31/2022
Trip Seconds,6`380`960,1`198.21,1`895.66,0,86`341
Trip Miles,6`382`369,6.19,8.00,0,2`967.54
Pickup Census Tract,2`623`831,17`031`468`160.38,368`945.90,17`031`010`100,17`031`980`100
Dropoff Census Tract,2`675`331,17`031`411`846.86,345`773.49,17`031`010`100,17`031`980`100
Pickup Community Area,5`868`572,32.35,25.20,1,77
Dropoff Community Area,5`748`741,25.84,20.93,1,77


In [26]:
val = int(dr.transpose()['count'].astype(float).max())

In [27]:
print(pds.gvf(val))

6'382'425


In [28]:
%%time
row_count = taxi.count()

# time cost i5 (7g), 4 workers (x) 2gb RAM
# CPU times: user 5.85 ms, sys: 871 µs, total: 6.72 ms
# Wall time: 11.8 s

CPU times: user 12.5 ms, sys: 2.31 ms, total: 14.8 ms
Wall time: 30.6 s


In [29]:
print(pds.gvf(row_count))

6'382'425


### Пандас - cпособы преобразования object столбца

In [30]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование to_numeric
drt = dr.transpose()
drtf = drt[pd.to_numeric(drt['count'], errors='coerce') > row_count*0.99][['count']]
# отображение отформатированных данных
pds.get_df_formated(drtf,'`',2, 10)
print('==================',pds.gvf(row_count))
drtf

================== 6'382'425


summary,count
Trip ID,6`382`425
Taxi ID,6`382`425
Trip Start Timestamp,6`382`425
Trip End Timestamp,6`382`213
Trip Seconds,6`380`960
Trip Miles,6`382`369
Fare,6`378`889
Tips,6`378`889
Tolls,6`378`889
Extras,6`378`889


In [31]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование astype
# 
# можно сразу применять тип и фильтровать, но значения в поле не поменяют тип
drtf1 = dr.transpose()[dr.transpose()['count'].astype(int) > -1][['count']]
# 
# если нужны вычисления с полем, то надо создать дф
drt1 = dr.transpose()
drt1['count'] = drt1['count'].astype(float)
drt1['not_null']=drt1['count']/row_count
# drt1[['count','not_null']]

In [32]:
drtf1 = drt1[drt1['count'] > -1][['count','not_null']]
# display(drtf1)
# отображение отформатированных данных
pds.get_df_formated(drtf1,'`',2, 10)
display(drtf1)
print('================================')
print('==total rows count==',pds.gvf(row_count))
print('================================')
print('=== columns count======',int(len(drtf1)))
print('================================')

summary,count,not_null
Trip ID,6`382`425,1
Taxi ID,6`382`425,1
Trip Start Timestamp,6`382`425,1
Trip End Timestamp,6`382`213,1.00
Trip Seconds,6`380`960,1.00
Trip Miles,6`382`369,1.00
Pickup Census Tract,2`623`831,0.41
Dropoff Census Tract,2`675`331,0.42
Pickup Community Area,5`868`572,0.92
Dropoff Community Area,5`748`741,0.90


==total rows count== 6'382'425
=== columns count====== 23


### take. Отбор только значений элементов строк (из объектов Row)

In [33]:
%%time
taxi.take(1)

CPU times: user 16.4 ms, sys: 2.97 ms, total: 19.4 ms
Wall time: 391 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)]

In [34]:
taxi.take(3)[:2]

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12', Taxi ID='449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='

In [35]:
%%time
# Отбор значений для одного элемента объекта Row (одно поле)
[row[ct.trip_i] for row in taxi.take(3)[:2]]

CPU times: user 9.91 ms, sys: 6.43 ms, total: 16.3 ms
Wall time: 422 ms


['bcfa19f2539021c054809d4c3993d226996ae095',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12']

In [36]:
# список словарей
[{field: row[field] for field in [ct.trip_i, ct.taxi_i]} for row in taxi.take(3)[:2]]

[{'Trip ID': 'bcfa19f2539021c054809d4c3993d226996ae095',
  'Taxi ID': '368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334'},
 {'Trip ID': '2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  'Taxi ID': '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317'}]

In [37]:
# список списков для нескольких полей, несколько проходов
[[row[ct.trip_i] for row in taxi.take(3)[:2]], [row[ct.taxi_i] for row in taxi.take(3)[:2]]]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [38]:
# список кортежей, один проход
list(zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]]))

[('bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 ('368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317')]

In [39]:
# список списков - один проход
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [40]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

### Использование zip(*...) и генератора списка

***Использование `zip(*...)` и генератора списка*** для преобразования кортежей в списки имеет минимальные накладные расходы. Эти операции довольно быстры, так как они выполняются в памяти и не требуют значительных вычислительных ресурсов.

Вот краткий обзор того, как работают эти преобразования:

**zip(*...)**: Это операция, которая транспонирует список списков. Она создает итератор, который группирует элементы из каждого из входных списков. Это делается за один проход по данным, что делает его очень эффективным.
Генератор списка: Преобразование каждого кортежа в список с помощью генератора списка `[list(t) for t in ...]` также выполняется за один проход по данным и выполняет простое преобразование типов. Это эффективно и не требует значительных вычислительных затрат.  
Для небольших наборов данных (несколько десятков или сотен элементов) накладные расходы будут практически незаметными. Для больших наборов данных, таких как миллионы элементов, накладные расходы могут стать более заметными, но по-прежнему будут минимальными по сравнению с другими более затратными операциями (например, ввод-вывод на диск или сложные вычисления).

Чтобы продемонстрировать минимальные накладные расходы, давайте рассмотрим пример с измерением времени:

In [41]:
# Пример данных
rows = [{ct.trip_i: f'trip_id_{i}', ct.taxi_i: f'taxi_id_{i}'} for i in range(1000000)]

# Измеряем время выполнения преобразования
start_time = time.time()
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in rows])]
end_time = time.time()

print("Время выполнения: ", end_time - start_time)

Время выполнения:  5.546215295791626


В большинстве случаев такие преобразования в Python очень эффективны, и вам не стоит беспокоиться о них, если нет необходимости оптимизировать производительность до предела.

### select. Отбор значений строк

In [42]:
taxi.select(ct.trip_i).take(4)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

In [43]:
taxi.select(ct.trip_i).take(4)[1:3]

[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace')]

In [44]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] \
                        for row in taxi.select(ct.trip_i,ct.taxi_i).take(4)[1:3]])]

[['2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  '54d812a0b88f8f9707825261014b3563a0a60ace'],
 ['449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317',
  'f98ae5e71fdda8806710af321dce58002146886c013f411be2cd336e0c224e78ed85626eeee462f0d1f5e6b29a030514b95f7b1b8ca59888a10b52dfe55ddf99']]

### Отбор строк через фильтрацию по номерам

#### monotonically_increasing_id для ДФ

In [45]:
from pyspark.sql.functions import monotonically_increasing_id

In [46]:
%%time
taxi.select(ct.trip_i).filter((monotonically_increasing_id() >= 1) \
                              & (monotonically_increasing_id() <= 4)).collect()

CPU times: user 33 ms, sys: 10.7 ms, total: 43.7 ms
Wall time: 39.3 s


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37'),
 Row(Trip ID='040caea96573c5743668b138011bc157c7825e86')]

#### zipWithIndex().filter для РДД

In [47]:
%%time
taxi.select(ct.trip_i).rdd.zipWithIndex().filter(lambda x: 1 <= x[1] <= 4).map(lambda x: x[0]).collect()

CPU times: user 134 ms, sys: 24.5 ms, total: 158 ms
Wall time: 1min 56s


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37')]

In [48]:
%%time
taxi.select(ct.trip_i).rdd.take(4)

CPU times: user 15.8 ms, sys: 1.1 ms, total: 16.9 ms
Wall time: 1.36 s


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

#### Причины неэффективности RDD Approach

**Монолитический подход DataFrame API:**  
`DataFrame API` в Apache Spark значительно оптимизирован за счет планировщика `Catalyst` и исполняющего механизма `Tungsten`. Эти оптимизации отсутствуют в `RDD API`, что делает операции на RDD в целом менее эффективными.  
`DataFrame API` оптимизирован с использованием `Catalyst optimizer`. Он понимает логику операций и может оптимизировать выполнение, используя проекцию, фильтрацию, и другие оптимизации.
В первом подходе, используя `monotonically_increasing_id, Spark` может оптимизировать фильтрацию на основе индексов и выполнять ее более эффективно.  
**Отсутствие оптимизации в RDD API:**  
`RDD API` не имеет уровня оптимизации, как `DataFrame API`. При использовании RDD, вы обязаны выполнять множество шагов вручную, что делает код менее эффективным.  
В данном случае, `zipWithIndex` создает пару `(элемент, индекс)`, которая потом фильтруется и мапится обратно. Это добавляет значительные накладные расходы на распределенные вычисления и коммуникацию между узлами кластера.  
**Сериализация и Десериализация:**  
Операции с RDD включают сериализацию и десериализацию данных между узлами, что может увеличить время выполнения, особенно на больших наборах данных.

**Как улучшить производительность RDD**  
Хотя `DataFrame API` чаще всего предпочтительнее для большинства операций из-за своих оптимизаций, бывают случаи, когда RDD необходимы. Вот несколько советов, чтобы улучшить производительность RDD:  
Используйте `Partitions Wisely`:  
Убедитесь, что количество партиций соответствует размерам данных и ресурсам кластера.  
`Persistence`:  
Используйте методы persist() или cache() для данных, которые будут многократно использоваться.  
`Avoid Shuffling`:  
Минимизируйте количество операций, требующих перемещения данных между узлами (например, groupByKey, reduceByKey).

**Добавление индексов:**  
`zipWithIndex()` добавляет индексы **к каждому элементу** в RDD. Эта операция может потребовать **полной загрузки** данных и выполнения дополнительного шага для распределения индексов.
**Фильтрация:**  
`filter(lambda x: 1 <= x[1] <= 4)` фильтрует элементы на основе их индексов. Эта операция применима **к каждому элементу** после добавления индексов.

Без `zipWithIndex()` не обойтись, если нужно отбирать элементы по их порядковому номеру, так как RDD по своей природе неупорядочен. Однако, если использовать take и затем отбирать нужные элементы, можно обойтись меньшими затратами.

#### rdd.take(4)[1:]

In [49]:
%%time
taxi.select(ct.trip_i).rdd.take(4)[1:]

CPU times: user 31.5 ms, sys: 253 µs, total: 31.7 ms
Wall time: 741 ms


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

#### limit(5).collect()[1:5]

In [50]:
taxi.select(ct.trip_i).limit(5).collect()[1:5]

[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37')]

### DF API и SQL - cравнение скорости выполнения запросов

In [51]:
%%time
pck = taxi.select(ct.pickup_ct).distinct().collect()

CPU times: user 28.1 ms, sys: 19.7 ms, total: 47.8 ms
Wall time: 43.7 s


In [52]:
%%time
taxi.createOrReplaceTempView("taxis")
query = "SELECT DISTINCT `Pickup Census Tract` FROM taxis"
result = spark.sql(query)
pck2 = result.collect()

CPU times: user 44.5 ms, sys: 2.79 ms, total: 47.3 ms
Wall time: 38.4 s


In [53]:
# Количество испытаний
num_trials = 5

# Время выполнения для первого блока кода: сбор уникальных значений с помощью DataFrame API
pck_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    pck = taxi.select('Pickup Census Tract').distinct().collect()
    end_time = time.time()
    pck_times.append(end_time - start_time)

# Вывод среднего времени выполнения для первого блока кода
print("Среднее время выполнения для первого блока кода: {:.2f} секунд".format(sum(pck_times) / num_trials))

100%|██████████| 5/5 [03:11<00:00, 38.27s/it]

Среднее время выполнения для первого блока кода: 38.27 секунд


In [54]:
# Время выполнения для второго блока кода: сбор уникальных значений с помощью SQL запроса
result_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    taxi.createOrReplaceTempView("taxis")
    result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
    pck2 = result.collect()
    end_time = time.time()
    result_times.append(end_time - start_time)

# Вывод среднего времени выполнения для второго блока кода
print("Среднее время выполнения для второго блока кода: {:.2f} секунд".format(sum(result_times) / num_trials))


100%|██████████| 5/5 [03:12<00:00, 38.52s/it]

Среднее время выполнения для второго блока кода: 38.51 секунд


**DF API по скорости сравнимо с SQL**. При этом SQL выполняется немного быстрее.

### filter. Несколько условий.

#### Разная сортировка: orderBy(taxi[f2].asc(), taxi[f3].desc())

In [55]:
%%time
f2 = 'Pickup Centroid Latitude'
f3 = 'Pickup Centroid Longitude'
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNotNull()).distinct().orderBy(taxi[f2].asc(), taxi[f3].desc()).collect()

CPU times: user 79.6 ms, sys: 12.1 ms, total: 91.7 ms
Wall time: 47 s


In [56]:
geos_pck_not_null[:5]

[Row(Pickup Centroid Latitude=41.651921576, Pickup Centroid Longitude=-87.564929171),
 Row(Pickup Centroid Latitude=41.660136051, Pickup Centroid Longitude=-87.60284764),
 Row(Pickup Centroid Latitude=41.663670652, Pickup Centroid Longitude=-87.540935513),
 Row(Pickup Centroid Latitude=41.663759731, Pickup Centroid Longitude=-87.637304085),
 Row(Pickup Centroid Latitude=41.665167734, Pickup Centroid Longitude=-87.535284961)]

#### Несколько условий. isNull(). isNotNull()

In [57]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNull()).collect()

CPU times: user 8.37 s, sys: 2.09 s, total: 10.5 s
Wall time: 1min 24s


In [58]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() & taxi[f3].isNull())

CPU times: user 280 ms, sys: 8.11 ms, total: 288 ms
Wall time: 421 ms


In [59]:
%%time
print(pds.gvf(geos_pck_not_null.count()))

511'551
CPU times: user 14.6 ms, sys: 890 µs, total: 15.5 ms
Wall time: 34.2 s


In [60]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNotNull()).count()

CPU times: user 25 ms, sys: 2.17 ms, total: 27.1 ms
Wall time: 34.7 s


0

In [61]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNull()).count()

CPU times: user 14.4 ms, sys: 13.2 ms, total: 27.7 ms
Wall time: 35.2 s


0

#### taxi[ct.trip_m]>10

In [62]:
taxi.filter(taxi[ct.trip_m]>10).count()

1777033

In [63]:
long = taxi.filter((taxi[ct.trip_m]>9.9) & (taxi[ct.trip_m]<10.1) 
                   & taxi[ct.pickup_cl].isNotNull() & taxi[ct.dropoff_cl].isNotNull() 
                   & taxi[ct.pickup_clon].isNotNull() & taxi[ct.dropoff_clon].isNotNull()
                  )

In [64]:
long.count()

20855

## withColumn

### вычисления на столбцах

In [65]:
long = long.withColumn('Distance',(
                                ((taxi[ct.pickup_cl]-taxi[ct.dropoff_cl])**2 +
                                (taxi[ct.pickup_clon]-taxi[ct.dropoff_clon])**2))**0.5)

In [66]:
%%time
long.select(ct.pickup_cl,ct.dropoff_cl,ct.pickup_clon,ct.dropoff_clon,'Distance').take(10)

CPU times: user 11.1 ms, sys: 20.7 ms, total: 31.8 ms
Wall time: 392 ms


[Row(Pickup Centroid Latitude=41.901206994, Dropoff Centroid Latitude=41.901206994, Pickup Centroid Longitude=-87.676355989, Dropoff Centroid Longitude=-87.676355989, Distance=0.0),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.663416405, Distance=0.11270646238477244),
 Row(Pickup Centroid Latitude=41.899602111, Dropoff Centroid Latitude=41.839086906, Pickup Centroid Longitude=-87.633308037, Dropoff Centroid Longitude=-87.714003807, Distance=0.10086573913913337),
 Row(Pickup Centroid Latitude=41.857183858, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.620334624, Dropoff Centroid Longitude=-87.663416405, Distance=0.1365046798204703),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.857183858, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.620334624, Distance=0.046343595040167315),
 Row(Pickup Centroid Latitude=41

### метод expr()

в PySpark можно использовать метод `expr()` внутри `withColumn()`. Метод `expr()` позволяет вам писать SQL-подобные выражения для создания нового столбца. Это особенно полезно, когда выражение сложное и его легче записать в виде строки.

In [ ]:
# Пример DataFrame
data = [("Alice", 10), ("Bob", 20), ("Charlie", 30)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Добавление нового столбца с использованием expr
df_with_new_column = df.withColumn("Age_Category", expr(
    "CASE WHEN Age < 20 THEN 'Teenager' " +
    "WHEN Age >= 20 AND Age < 30 THEN 'Young Adult' " +
    "ELSE 'Adult' END"
))
.withColumn("Age_In_Decades", expr("round(float(Age/10), 2)"))

# Отображение результата
df_with_new_column.show()

### Метод .getItem()

Метод `.getItem()` в PySpark используется для доступа к элементам в столбцах типа `ArrayType`, `MapType`, или `StructType`.

#### Доступ к элементу в массиве (списке)

In [ ]:
# Пример DataFrame с массивами
data = [("Alice", [1, 2, 3]), ("Bob", [4, 5, 6])]
df = spark.createDataFrame(data, ["Name", "Numbers"])

# Доступ к первому элементу массива
df_with_first_number = df.withColumn("First_Number", col("Numbers").getItem(0))

```yaml
+-----+---------+------------+
|Nam e|  Numbers|First_Number|
+-----+---------+------------+
|Alice|[1, 2, 3]|           1|
|  Bob|[4, 5, 6]|           4|
-+----+---------+------------+


#### Доступ к элементу в словаре (хэшмэп)

In [ ]:
# Пример DataFrame с картами
data = [("Alice", {"math": 85, "science": 90}), ("Bob", {"math": 78, "science": 88})]
df = spark.createDataFrame(data, ["Name", "Scores"])

# Доступ к значению по ключу
df_with_math_score = df.withColumn("Math_Score", col("Scores").getItem("math"))

df_with_math_score.show()

```yaml
+-----+---------------------------+----------+
| Name|                     Scores|Math_Score|
+-----+---------------------------+----------+
|Alice|{math -> 85, science -> 90}|        85|
|  Bob|{math -> 78, science -> 88}|        78|
+-----+---------------------------+----------+


#### Доступ к элементу в структуре

In [ ]:
# Создание схемы для поля Address
address_schema = StructType([
    StructField("City", StringType(), True),
    StructField("ZIP", IntegerType(), True)
])

# Создание основной схемы
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Address", address_schema, True)
])

# Пример данных
data = [("Alice", ("New York", 10001)), ("Bob", ("San Francisco", 94105))]

# Создание DataFrame с использованием схемы
df = spark.createDataFrame(data, schema=schema)

# Доступ к полю структуры и создание нового столбца
df_with_city = df.withColumn("City", col("Address").getItem("City"))
df_with_city.show(truncate=False)

```yaml
+----+-----------------------+-------------+
|Name|Address                |City         |
+----+-----------------------+-------------+
|Alice|{New York, 10001}     |New York     |
|Bob  |{San Francisco, 94105}|San Francisco|
+----+-----------------------+-------------+


### .getItem() в select, filter, where, и groupBy

Метод `.getItem()` чаще всего используется в контексте `withColumn` для создания нового столбца на основе элементов массива, карты или структуры в существующем столбце. Однако его также можно использовать в других методах, таких как `select, filter, where, и groupBy`, когда требуется доступ к элементам сложных типов данных.

In [ ]:
# Пример DataFrame с массивами
data = [("Alice", [1, 2, 3]), ("Bob", [4, 5, 6]), ("Charlie", [1, 5, 9])]
df = spark.createDataFrame(data, ["Name", "Numbers"])

#### Использование с withColumn

In [ ]:
df_with_first_number = df.withColumn("First_Number", col("Numbers").getItem(0))
df_with_first_number.show()

```yaml
+-------+---------+------------+
|   Name|  Numbers|First_Number|
+-------+---------+------------+
|  Alice|[1, 2, 3]|           1|
|    Bob|[4, 5, 6]|           4|
|Charlie|[1, 5, 9]|           1|
+-------+---------+------------+


#### Использование с select

In [ ]:
df_selected = df.select(col("Name"), col("Numbers").getItem(0).alias("First_Number"))
df_selected.show()

```yaml
+-------+------------+
|   Name|First_Number|
+-------+------------+
|  Alice|           1|
|    Bob|           4|
|Charlie|           1|
+-------+------------+

#### Использование с filter или where

In [ ]:
df_filtered = df.filter(col("Numbers").getItem(0) > 2)
df_filtered.show()

```yaml
+----+---------+
|Name|  Numbers|
+----+---------+
| Bob|[4, 5, 6]|
+----+---------+


#### Использование с groupBy

In [ ]:
df_grouped = df.groupBy(col("Numbers").getItem(0)).count()
df_grouped.show()

```yaml
+----------+-----+
|Numbers[0]|count|
+----------+-----+
|         1|    2|
|         4|    1|
------+----+-----+


## UDF

#### UDF для строк и столбцов

В PySpark, UDF (User Defined Function) применяется к значениям того ряда DataFrame, в котором она вызвана. UDF работает на уровне отдельных строк, и каждая строка передаётся в UDF по отдельности. Таким образом, UDF применяется только к данным текущего ряда, когда она вызывается, и не имеет прямого доступа к значениям других рядов.

Если вы хотите использовать UDF, и вам нужно работать с временными рядами или выполнять операции, которые зависят от значений нескольких строк, вам придётся преобразовать данные таким образом, чтобы все необходимые значения были доступны внутри одной строки. Это может быть сделано с помощью оконных функций или других механизмов для агрегации данных.

Преобразование столбца временных рядов в строку DataFrame — это не всегда оптимальное решение и может быть воспринято как хардкод, особенно если данные имеют сложную структуру или большой объём. Однако, есть способы, чтобы сделать это более гибко и масштабируемо.

In [ ]:
data = [
    ("2021-01-01", 100),
    ("2021-01-02", 200),
    ("2021-01-03", 300),
    ("2021-01-04", 400)
]

columns = ["date", "value"]

df = spark.createDataFrame(data, columns)

In [ ]:
# Определение окна
window_spec = Window.orderBy("date")

# Использование оконной функции lag для получения предыдущего значения
df_with_lag = df.withColumn("previous_value", lag(col("value"), 1).over(window_spec))

In [ ]:
# Определение UDF
def my_udf(current_value, previous_value):
    if previous_value is None:
        return 0
    return current_value - previous_value

my_udf = udf(my_udf, IntegerType())

# Применение UDF к DataFrame
df_with_udf = df_with_lag.withColumn("difference", my_udf(col("value"), col("previous_value")))

df_with_udf.show()

Вывод
```yaml
+----------+-----+--------------+----------+
|      date|value|previous_value|difference|
+----------+-----+--------------+----------+
|2021-01-01|  100|          null|         0|
|2021-01-02|  200|           100|       100|
|2021-01-03|  300|           200|       100|
|2021-01-04|  400|           300|       100|
+----------+-----+--------------+----------+
```

Мы определили оконную спецификацию, чтобы упорядочить строки по дате.  
С помощью оконной функции lag мы получили значение предыдущей строки и добавили его как новый столбец previous_value.  
Определили UDF, которая вычисляет разность между текущим значением и предыдущим значением.  
Применили UDF к DataFrame, добавив новый столбец difference, содержащий результаты вычислений.

In [ ]:
data = [
    (1, [0.1, 0.2, 0.3, 0.4]),
    (2, [0.2, 0.3, 0.4, 0.5]),
    (3, [0.3, 0.4, 0.5, 0.6])
]

columns = ["id", "time_series"]

df = spark.createDataFrame(data, columns)

In [ ]:
def welch_udf(time_series):
    # Преобразование списка в numpy массив
    time_series_np = np.array(time_series)
    # Применение функции welch
    f, Pxx = welch(time_series_np)
    # Возвращение среднего значения спектральной плотности мощности как пример
    return float(np.mean(Pxx))

# Регистрация UDF
welch_udf = udf(welch_udf, FloatType())

df_with_welch = df.withColumn("welch_result", welch_udf(df["time_series"]))
df_with_welch.show()

Работа с временными рядами большого объёма, такими как, например, ежечасные данные за год для 700 районов города (что эквивалентно более чем 6 миллионам точек данных), может представлять значительные вычислительные и ресурсные сложности. Оборачивание этих временных рядов в строки и использование UDF может быть ограничивающим из-за:

- Памяти: При работе с такими большими временными рядами может возникнуть значительное потребление памяти, так как каждая строка будет содержать массив данных за год.
- Производительности: Использование UDF в PySpark на больших объёмах данных может быть медленным, особенно если UDF выполняет сложные вычисления.

Возможные варианты решения:

**Использование Оконных Функций и Группировка:**  
- Вместо оборачивания данных в строки, можно использовать оконные функции для обработки временных рядов.  
Например, можно разбить данные на более мелкие временные окна (недели или месяцы) и применять функции к этим окнам, чтобы снизить нагрузку на память и улучшить производительность.  

**Пакетная Обработка и Декомпозиция:**  
- Разделить данные на меньшие, более управляемые части.  
- Параллельно обрабатывать каждую часть и затем агрегировать результаты.  
- Это поможет уменьшить нагрузку на память и улучшить производительность.  

**Оптимизация UDF:** 
- Убедитесь, что UDF оптимизирована для работы с большими объёмами данных.  
- Используйте библиотеки, которые поддерживают векторизацию и многопоточность.  

**Использование Специализированных Инструментов:**
- Рассмотрите возможность использования специализированных библиотек для обработки временных рядов, таких как sktime или tsfresh, которые могут быть интегрированы с PySpark.  
- Эти библиотеки могут предоставить более эффективные методы обработки временных рядов.  

**Снижение Разрешения Данных:**  
- Если допустимо, можно снизить временное разрешение данных (например, перейти от ежечасных данных к ежедневным).  
- Это существенно уменьшит объём данных и упростит обработку.  

**Распределённые Вычисления:**
- Использование возможностей PySpark для распределённых вычислений.  
- Убедитесь, что ваш кластер настроен правильно для обработки больших объёмов данных, включая достаточные ресурсы памяти и процессоров.  

#### Пример UDF (с устаревшим способом указания типа UDF)
для возврата значения функции норм.распределения вер-ти для 1М случайных значений

In [67]:
%%time

from scipy import stats

big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val',f.rand())
)

big_df.cache()
big_df.show(3)

@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))

(
    big_df
    .withColumn('probability', pandas_pdf(big_df.val))
    .show(5)
)

+---+-------------------+
| id|                val|
+---+-------------------+
|  0|0.15316890012395712|
|  1| 0.6708241117542864|
|  2| 0.1710217543507765|
+---+-------------------+
only showing top 3 rows



/usr/local/spark/python/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---+-------------------+-------------------+
| id|                val|        probability|
+---+-------------------+-------------------+
|  0|0.15316890012395712|0.39428988590544156|
|  1| 0.6708241117542864| 0.3185610866119611|
|  2| 0.1710217543507765| 0.3931505138903692|
|  3| 0.3144878389031438| 0.3796939165449698|
|  4| 0.5572331819593144| 0.3415733133118225|
+---+-------------------+-------------------+
only showing top 5 rows

CPU times: user 498 ms, sys: 27.2 ms, total: 525 ms
Wall time: 9.79 s


Предупреждение  
`/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn`  

связано с устареванием способа указания типа UDF (User Defined Function) в PySpark. Сообщение указывает на то, что начиная с Python 3.6 и Spark 3.0, предпочтительно использовать аннотации типов вместо указания типа UDF через параметр f.PandasUDFType.

Конкретно предупреждение означает, что метод указания типа UDF через f.PandasUDFType.SCALAR будет устаревшим в будущих версиях PySpark, и рекомендуется использовать аннотации типов для Pandas UDF. Это сделано для повышения ясности кода и его поддерживаемости

В `@f.pandas_udf('double', f.PandasUDFType.SCALAR)`
- первый параметр `'double'` указывает тип возвращаемого значения ЮДФ,
- второй параметр - это тип функции. Если возвращается один столбец (`.Series(...)`), используется `.PandasUDFType.SCALAR`, если возвращается несколько столбцов (pandas `DataFrame(...)`), `.PandasUDFType.GROUPED_MAP`

### Пример UDF (аргумент - Series) с аннотацией типов

In [68]:
%%time
from pyspark.sql.types import DoubleType

big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val', f.rand())
)

big_df.cache()
big_df.show(3)

@f.pandas_udf(DoubleType())
def pandas_pdf(v: pd.Series) -> pd.Series:
    return pd.Series(stats.norm.pdf(v))

result = big_df.withColumn('pdf', pandas_pdf(f.col('val')))
result.show(3)

+---+------------------+
| id|               val|
+---+------------------+
|  0|0.2316482258404361|
|  1|0.6999371995807026|
|  2|0.3270694747541646|
+---+------------------+
only showing top 3 rows

+---+------------------+-------------------+
| id|               val|                pdf|
+---+------------------+-------------------+
|  0|0.2316482258404361|0.38838079762633254|
|  1|0.6999371995807026|0.31226765982726706|
|  2|0.3270694747541646| 0.3781645903738832|
+---+------------------+-------------------+
only showing top 3 rows

CPU times: user 64.6 ms, sys: 25.9 ms, total: 90.5 ms
Wall time: 10.8 s


Здесь `DoubleType()` используется вместо `f.PandasUDFType.SCALAR`, и типы указаны непосредственно в аннотациях функции `(v: pd.Series) -> pd.Series`. Это улучшает читаемость кода и соответствует новым стандартам PySpark.

В актуальном способе указания типов для PySpark UDF для определения UDF, которая возвращает несколько столбцов (pandas DataFrame), вы можете использовать аннотацию функции и указание типа возвращаемого значения при помощи PySpark StructType. В этом случае вам понадобится указать схему (schema), описывающую структуру возвращаемого DataFrame.

In [ ]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

# Определяем схему возвращаемого DataFrame
schema = StructType([
    StructField("col1", DoubleType(), True),
    StructField("col2", StringType(), True)
])

# Определяем Pandas UDF
@pandas_udf(schema)
def my_udf(input_col: pd.Series) -> pd.DataFrame:
    # Ваш код для обработки DataFrame
    result = pd.DataFrame({
        "col1": input_col * 2,
        "col2": input_col.astype(str)
    })
    return result

# Создаем DataFrame
df = spark.createDataFrame(
    [(1.0,), (2.0,), (3.0,)],
    ['input_col']
)

# Применяем UDF к столбцу
df.select(my_udf(df['input_col'])).show()

Pandas UDF в PySpark может принимать несколько pandas Series объектов в качестве аргументов. Для этого необходимо использовать декоратор @pandas_udf и определить схему выходного DataFrame.

In [ ]:
# Определяем схему выходного DataFrame
output_schema = StructType([
    StructField('output_col1', DoubleType()),
    StructField('output_col2', DoubleType())
])

# Определяем Pandas UDF с использованием декоратора @pandas_udf
@pandas_udf(output_schema)
def my_udf(input_col1: pd.Series, input_col2: pd.Series) -> pd.DataFrame:
    # Создаем DataFrame из двух входных Series
    output_df = pd.DataFrame({
        'output_col1': input_col1 + input_col2,
        'output_col2': input_col1 * input_col2
    })
    return output_df

# Создаем DataFrame для теста
df = spark.createDataFrame(
    [(1.0, 2.0), (3.0, 4.0), (5.0, 6.0)],
    ['input_col1', 'input_col2']
)

# Применяем Pandas UDF к DataFrame
result_df = df.select(my_udf(df['input_col1'], df['input_col2']))
result_df.show()

### Пример UDF (аргумент - DataFrame) с аннотацией типов

Вы можете передать в UDF аргумент в виде DataFrame, но это будет работать только с UDF типа GROUPED_MAP. В этом случае каждый входной DataFrame будет представлять собой группу данных, по которой будет выполняться UDF. Для этого типа UDF вам необходимо определить схему входного DataFrame и схему выходного DataFrame.

In [ ]:
# Определяем схему входного DataFrame
input_schema = StructType([
    StructField("col1", DoubleType(), True),
    StructField("col2", DoubleType(), True)
])

# Определяем схему возвращаемого DataFrame
output_schema = StructType([
    StructField("col1", DoubleType(), True),
    StructField("col2", DoubleType(), True),
    StructField("col3", StringType(), True)
])

# Определяем Pandas UDF типа GROUPED_MAP
@pandas_udf(output_schema, f.PandasUDFType.GROUPED_MAP)
def my_udf(pdf: pd.DataFrame) -> pd.DataFrame:
    # Ваш код для обработки DataFrame
    pdf["col3"] = (pdf["col1"] + pdf["col2"]).astype(str)
    return pdf

# Создаем DataFrame
df = spark.createDataFrame(
    [(1.0, 2.0), (3.0, 4.0), (5.0, 6.0)],
    ['col1', 'col2']
)

# Применяем UDF к DataFrame с использованием groupBy и apply
# В данном случае мы используем groupBy().apply() даже без явного группирования
df.groupBy().apply(my_udf).show()

Возможное предупреждение  
`/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(`  
говорит о том, что используется устаревший способ `df.groupBy().apply(my_udf).show()`. Pandas UDFs с использованием `applyInPandas` предоставляют более современный и предпочтительный способ обработки pandas DataFrame.

Pandas UDF в режиме GROUPED_MAP не поддерживает передачу нескольких pandas DataFrame в качестве отдельных аргументов. Вместо этого вы должны объединить данные в один DataFrame до передачи в функцию. В режиме SCALAR Pandas UDF может принимать несколько Series в качестве аргументов.

Для использования нескольких pandas DataFrame в Pandas UDF с декоратором @pandas_udf в режиме GROUPED_MAP, вам нужно объединить два DataFrame в один перед тем, как вызывать функцию. PySpark Pandas UDF с типом GROUPED_MAP ожидает один pandas DataFrame в качестве аргумента

In [ ]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType

# Определяем схему выходного DataFrame
output_schema = StructType([
    StructField('id', IntegerType()),
    StructField('output_col1', DoubleType()),
    StructField('output_col2', DoubleType())
])

# Определяем Pandas UDF с использованием декоратора @pandas_udf
@pandas_udf(output_schema, f.PandasUDFType.GROUPED_MAP)
def my_udf(pdf: pd.DataFrame) -> pd.DataFrame:
    # Создаем выходной DataFrame
    output_df = pd.DataFrame({
        'id': pdf['id'],
        'output_col1': pdf['input_col1'] + pdf['input_col2'],
        'output_col2': pdf['input_col1'] * pdf['input_col2']
    })
    return output_df

# Создаем два DataFrame для теста
df1 = spark.createDataFrame(
    [(1, 1.0), (2, 3.0), (3, 5.0)],
    ['id', 'input_col1']
)

df2 = spark.createDataFrame(
    [(1, 2.0), (2, 4.0), (3, 6.0)],
    ['id', 'input_col2']
)

# Объединяем два DataFrame по столбцу 'id'
df = df1.join(df2, on='id')

# Применяем Pandas UDF к DataFrame с использованием groupBy
result_df = df.groupBy('id').apply(my_udf)
result_df.show()

### Сравнение производительности векторизированой и не_ ЮДФ

In [ ]:
big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val',f.rand())
)

In [ ]:
def test_pandas_pdf():
    return (
        big_df
        .withColumn('probability', pandas_pdf(big_df.val))
        .agg(f.count(f.col('probability')))
        .show()
    )

%timeit -n 1 test_pandas_pdf()

In [ ]:
#row-by-row version with Python-JVM conversion
@f.udf('double')
def pdf(v):
    return float(stats.norm.pdf(v))

def test_pdf():
    return (
        big_df
        .withColumn('probability', pdf(big_df.val))
        .agg(f.count(f.col('probability')))
        .show()
    )

%timeit -n 1 test_pdf()

Результат для векторизированной ЮДФ - 903 ms, против не_ - 34.4 s

Отличие примерно в 40 раз!

### Использование объектов pandas в Pandas UDF

В **Pandas UDF** (User Defined Function) с использование декоратора `@pandas_udf` в Spark DataFrame API, функция pandas_pdf принимает один или несколько pandas Series (DataFrame) объектов и возвращает pandas Series (DataFrame) объект. В этом случае, pandas_pdf принимает Series v (который представляет собой значения из столбца val в DataFrame) и возвращает Series, содержащий значения, вычисленные с использованием функции stats.norm.pdf().

Возвращая `pd.Series(stats.norm.pdf(v))`, мы создаем объект Pandas Series, который содержит значения, вычисленные с помощью функции `stats.norm.pdf()`, примененные к каждому элементу входного Series v. Это необходимо потому, что Spark использует PyArrow для обмена данными между JVM и Python, и PyArrow не может обрабатывать объекты `stats.norm.pdf(v)` напрямую, так как они не являются объектами pandas.

Поэтому, оборачивая результат stats.norm.pdf(v) в pd.Series(), мы создаем объект pandas, который Spark может обработать, и возвращаем его из функции pandas_pdf.

## .cache() .unpersist() - кэш памяти

Когда вы вызываете `big_df.cache()`, вы помещаете DataFrame big_df в **кэш памяти**. Это означает, что DataFrame будет кэширован в памяти и будет доступен для быстрого доступа к данным при последующих операциях.

Однако, если у вас есть ограничения на использование памяти или если кэш становится необходимым для других операций, вы можете явно удалить кэш с помощью метода `unpersist()`.

Это удалит кэш из памяти и освободит ресурсы, занимаемые DataFrame big_df. Если вы не удаляете кэш явно с помощью unpersist(), DataFrame останется в кэше и будет занимать память до тех пор, пока не будет удален Spark контекст или пока явно не будет вызвана операция `unpersist()`

In [ ]:
big_df.unpersist()

## Apache Arrow Memory & runtime switch

До версии ПиСпарк 2.3 использование ЮДФ в Питоне проигрывало по производительности Скале и Джаве (из-за переключений времени выполнения `runtime` между Питон и JVM).

С версии 2.3 Спарк начал использовать **Apache Arrow Memory** - общее пространство памяти для разных сред окружения, поэтому отпала необходимость и расходны на конвертацию объектов.

Также Arrow хранит в памяти **столбчатые объекты** (columnar objects), что повышает производительность. Используя это, ПиСпарк отрефакторилился, в результате чего мы получили **векторизированные ЮДФ**.

В аспектах программирования и операционных систем (в частности, процессов и потоков), термины "runtime" и "switch runtime" имеют разные значения. Давайте рассмотрим их подробнее:

**Runtime**  
**Программирование:**  
`Runtime` (время выполнения) - это фаза, когда программа исполняется. В это время происходят все динамические операции, такие как выполнение кода, управление памятью, обработка исключений и т.д. Например, в Python есть интерпретатор, который выполняет код во время исполнения программы.
`Runtime Environment` (среда выполнения) - это окружение, в котором исполняется программа. Оно включает в себя библиотеки, интерпретаторы (например, `Python Interpreter`) или виртуальные машины (например, `Java Virtual Machine`).  
**Процессы и потоки ОС:**  
`Runtime` в контексте ОС может также относиться к продолжительности времени, когда процесс или поток активен и выполняется на процессоре. Это время исполнения может включать в себя время, проведенное на процессоре, и время ожидания (например, ожидание ввода-вывода).

**Switch Runtime**  
**Программирование:**  
`Switch Runtime` (переключение времени выполнения) в программировании часто означает смену среды выполнения для кода. Например, если вы хотите исполнять код, написанный на Python, в среде JVM, это может потребовать использование специального инструмента или транслятора, который позволит исполнять Python-код на Java Virtual Machine (например, Jython).  
**Процессы и потоки ОС:**  
`Context Switch` (переключение контекста) - это процесс, при котором операционная система переключается от выполнения одного процесса или потока к другому. Это необходимо для многозадачности, чтобы несколько процессов могли совместно использовать ресурсы процессора.
В данном контексте, "switch runtime" может подразумевать переключение между разными процессами или потоками в ходе их выполнения. То есть операционная система переключает время выполнения между различными задачами для обеспечения параллелизма и эффективного использования процессора.

**Примеры**  
**Программирование:**  
Вы используете Python для разработки, но вам нужно интегрировать код с существующим приложением на Java. Вы используете Jython для "switch runtime" и исполняете Python-код в JVM.
**Процессы и потоки ОС:**  
В многозадачной операционной системе несколько программ работают одновременно. Процессор переключается между задачами (процессами) каждые несколько миллисекунд, обеспечивая иллюзию параллельного выполнения. Это и есть "context switch".

**Резюме**  
`Runtime` (время выполнения): в программировании - фаза исполнения программы; в ОС - время, когда процесс или поток активно выполняется.
`Switch Runtime` (переключение времени выполнения): в программировании - смена среды выполнения для кода; в ОС - переключение контекста между процессами или потоками для многозадачности.

## Column объекты.

### Использование объектов

In [ ]:
data = [("John Doe", 30), ("Jane Smith", 25), ("Bob Johnson", 35)]
columns = ["full name", "age"]

# Создание DataFrame
df = spark.createDataFrame(data, schema=columns)

# Обращение к полям через точку
df.show()
df.select("full name").show()
print(df["full name"])
print(df.age)
df.age.show()

Для инструкции `df.age.show()` получим ошибку TypeError: 'Column' object is not callable. Обращение к столбцам через точку, как в df.age, может вызвать ошибку TypeError, так как это возвращает объект Column, который не является вызываемым (callable) для метода show().

Объекты `df.age` и `df["full name"]` в PySpark являются объектами типа Column. Column - это абстракция, представляющая столбец данных в DataFrame. Объект Column обеспечивает различные методы для выполнения операций на столбцах данных, таких как вычисления, фильтрация, агрегация и т. д.

Чтобы использовать объект Column, вы можете использовать его методы для создания новых столбцов, применения функций к существующим столбцам и других операций с данными в DataFrame. Например, вы можете использовать объект Column для создания нового столбца, содержащего возраст пассажиров в годах путем деления столбца "age" на 10.

In [ ]:
df = df.withColumn("age_in_decades", df.age/10)
df.show()

Или можете отфильтровать датафрейм по полю

In [ ]:
df.filter(df.age_in_decades > 3).show()

### Обращение к столбцам через точку

Обращение к полям DataFrame в ПиСпарк через точку, если имена полей содержат несколько слов через пробел, невозможно - необходимо использовать квадратные скобки и строковые индексы.

### Функция col

Функция `col` в PySpark используется для указания на столбцы в DataFrame. Это удобно, когда вы хотите ссылаться на столбцы программным способом, особенно в выражениях и операциях DataFrame.

```python
df.select(col("columnName"))

```
Здесь `col("columnName")` то же самое, что и `df["columnName"]` - объект Column

В большинстве случаев инструкции `df.select(col("columnName"))` и `df.select("columnName")` ведут себя одинаково и возвращают один и тот же результат.

С `col` вы можете легко создавать более **сложные выражения**.
```python
df.select((col("columnName") + 1).alias("newColumnName"))

```

При использовании `alias` (псевдонимов таблиц) `col` позволяет вам указывать **полное имя столбца**.
```python
df.alias("a").select(col("a.columnName"))

```

В сложных выражениях и join-ах `col` может сделать **код более читаемым** и понятным.
```python
df1.join(df2, col("df1.columnName") == col("df2.columnName"))`

**Использование `col` полезно, когда**:  
- Вы работаете с более сложными выражениями.
- Вы работаете с несколькими DataFrame и хотите избежать неоднозначностей в именах столбцов.
- Вам нужно указать столбец программно, особенно в циклах или функциях.  

**Когда можно использовать строки напрямую**:  
Использование строк напрямую может быть удобным для простых операций выборки, когда вы выбираете один или несколько столбцов без сложных выражений.

**Если имя столбца в DataFrame содержит пробелы**, вам нужно будет использовать метод col и обернуть имя столбца в обратные кавычки ( \` ). Это необходимо для того, чтобы правильно идентифицировать такие имена столбцов в PySpark.  
```python
df1.join(df2, col("`df1.column Name`") == col("df2.columnName"))
```

In [ ]:
# Пример данных для df1 и df2
data1 = [("John", 28), ("Doe", 35)]
columns1 = ["column Name", "Age"]
df1 = spark.createDataFrame(data1, columns1)

data2 = [("John", "New York"), ("Doe", "Los Angeles")]
columns2 = ["columnName", "City"]
df2 = spark.createDataFrame(data2, columns2)

# Присвоение алиасов df1 и df2
df1_alias = df1.alias("a")
df2_alias = df2.alias("b")

# Выполнение join с использованием col и алиасов
result = df1_alias.join(df2_alias, col("a.`column Name`") == col("b.columnName"))

# Выбор столбцов для отображения
result.select(
    col("a.`column Name`").alias("column_Name"), 
    col("b.City")
).show()

Если вы используете **переменные для создания строкового имени столбца** (содержащего пробелы) и конкатенируете его, то вам не нужно дополнительно оборачивать имя столбца в обратные кавычки. PySpark корректно обработает такой синтаксис. Обратные кавычки необходимы только для строковых литералов, которые содержат пробелы или другие специальные символы.
```python
...
.select(
    f.col("b." + ct.pickup_ct), 
    f.col("b." + ct.pickup_cl), 
    f.col("b." + ct.pickup_clon)
)
```

полный пример в разделе про джойны

#### Динамическое использование 

**Динамическое использование** функции col в PySpark позволяет вам строить имена столбцов на лету, используя переменные или выражения.
Это особенно полезно, когда вы хотите написать универсальный код, который работает с различными именами столбцов в зависимости от условий или данных.



**Использование выражений**
```python
df.select(col("prefix_" + column_name))
```

**В циклах и функциях**  
Пример для подсчета пропущенных значенийЖ

In [ ]:
# Пример данных
data = [
    ("Alice", 25, "Math", "A"),
    ("Bob", None, "Physics", "B"),
    ("Charlie", 30, "History", ""),
    ("Diana", 22, "", "A"),
    ("Eve", None, "Math", "C"),
]

columns = ["Name", "Age", "Subject", "Grade"]
df = spark.createDataFrame(data, columns)

# Применяем код для подсчета пустых значений, NULL и NaN
df.select(
    [count(when(col(c)=='') | col(c).isNull() | isnan(c), c).alias(c) for c in df.columns]
).show()

```yaml
+-------+---+--------------+-----+
|   Name|Age|       Subject|Grade|
+-------+---+--------------+-----+
|      0|  2|             1|    1|
+-------+---+--------------+-----+--+


**Фильтрация по результатам агрегации**

In [ ]:
# Пример данных
data = [("John", "Sales"), ("Doe", "HR"), ("Alice", "Sales")]
columns = ["Name", "Department"]
df = spark.createDataFrame(data, columns)

# Фильтрация по количеству записей в группе "Department" больше 1
df.groupBy("Department").count().filter(col("count") > 1).show()

**Добавление вычисляемого столбца и его использование для фильтрации**

In [ ]:
# Пример данных
data = [("John", 28), ("Doe", 35)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# Добавление столбца "is_adult" и фильтрация по нему
df.withColumn("is_adult", when(col("Age") >= 18, "Adult").otherwise("Minor")).filter(col("is_adult") == "Adult").show()

## Джойны

Подготовим данные для примеров

In [ ]:
# ctp - pickup census tract
ctp = taxi.groupBy(ct.pickup_ct).count().orderBy("count", ascending=False).cache()
# ctd - dropoff census tract
ctd = taxi.groupBy(ct.dropoff_ct).count().orderBy("count", ascending=False).cache()н

В Apache Spark доступны следующие типы джойнов:

**inner:**  
Возвращает строки, которые имеют совпадающие значения в обеих таблицах.

In [ ]:
inner_join = ctd.join(ctp, ctd[ct.dropoff_ct] == ctp[ct.pickup_ct], 'inner')
inner_join.show()

In [ ]:
# inner join с проверкой совпадения по двум столбцам
inner_join = ctd.join(ctp, (ctd[ct.dropoff_ct] == ctp[ct.pickup_ct]) & \
                      (ctd['count'] == ctp['count']), 'inner')
inner_join.show()

**cross:**  
Возвращает декартово произведение двух таблиц, где каждая строка первой таблицы соединяется с каждой строкой второй таблицы.

In [ ]:
cross_join = ctd.crossJoin(ctp)

**outer, full, full_outer:**  
Возвращает все строки, когда есть совпадение в одной из таблиц. Включает строки, не имеющие совпадений из обеих таблиц.  
Методы outer, full, и full_outer в контексте Apache Spark DataFrame join фактически являются синонимами и выполняют одинаковую операцию. Они все возвращают полный внешний джойн (full outer join), что означает, что они возвращают все строки из обеих таблиц, когда есть совпадение по ключу, или строки с NULL, если нет совпадения.  
Три названия (outer, full, и full_outer) предоставляются для удобства и читаемости кода. Вот основные причины, почему может быть несколько названий для одной и той же операции:  
- Читаемость и предпочтения разработчиков. В разных системах и инструментах SQL могут использоваться разные термины для одной и той же операции. Предоставляя несколько вариантов, Spark обеспечивает совместимость с разными системами и инструментами, делая код более переносимым.
- Совместимость и переносимость. В некоторых контекстах или командах могут быть общепринятые стандарты или конвенции наименования. Spark предоставляет гибкость, чтобы разработчики могли следовать этим стандартам без необходимости адаптировать терминологию.
- Общепринятые стандарты.  некоторых контекстах или командах могут быть общепринятые стандарты или конвенции наименования. Spark предоставляет гибкость, чтобы разработчики могли следовать этим стандартам без необходимости адаптировать терминологию.

**Left, left_outer:** Возвращает все строки из левой таблицы и совпадающие строки из правой таблицы. Если нет совпадения, результаты правой таблицы будут NULL.

In [ ]:
left_join = ctd.join(ctp, ctd[ct.dropoff_ct] == ctp[ct.pickup_ct], 'left')
left_join.show()

**right, right_outer:** Возвращает все строки из правой таблицы и совпадающие строки из левой таблицы. Если нет совпадения, результаты левой таблицы будут NULL.

In [ ]:
right_join = ctd.join(ctp, ctd[ct.dropoff_ct] == ctp[ct.pickup_ct], 'right')
right_join.show()

**left_semi:** Возвращает только те строки из левой таблицы, для которых существуют совпадающие строки в правой таблице. В отличие от других типов джойнов, он не возвращает строки из правой таблицы.

In [ ]:
left_semi_join = ctd.join(ctp, ctd[ct.dropoff_ct] == ctp[ct.pickup_ct], 'left_semi')
left_semi_join.show()

**left_anti:** Возвращает только те строки из левой таблицы, для которых не существует совпадающих строк в правой таблице. Этот тип джойна используется для нахождения строк, которые существуют в левой таблице, но не имеют совпадений в правой таблице.

In [ ]:
left_anti_join = ctd.join(ctp, ctd[ct.dropoff_ct] == ctp[ct.pickup_ct], 'left_anti')
left_anti_join.show()

### Джойны: DF API vs SQL

Джойны с использованием АПИ датафрейма могут быть более громоздкими в плане синтаксиса из-за необходимости использования конструкций с псевдонимами (`alias`) и функций `col`

Подготовим таблицы для джойна

In [ ]:
# ДФ по районам посадки, содержащий тройки: код р-на, широта, долгота
ctp_geo = taxi.groupBy(ct.pickup_ct,ct.pickup_cl,ct.pickup_clon).count().orderBy("count", ascending=False)
ctp_geo.cache()
# Посмотрим повторяющиеся геоточки
ctp_geo_dupl = ctp_geo.groupBy(ct.pickup_cl,ct.pickup_clon).count().filter(f.col('count') > 1).cache()
ctp_geo_dupl.show()
# Отберем отдельно те из них, которые имеют непустое значение для кода района.
ctp_geo_nulls = ctp_geo.filter(ctp_geo[ct.pickup_cl].isNull() \
               & ctp_geo[ct.pickup_clon].isNull()\
               & ctp_geo[ct.pickup_ct].isNotNull())

Свяжем таблицы, используя ДФ АПИ

In [ ]:
# Cвяжем дубликаты с непустыми тройками (р-н, шир, долг)
# Также посмотрим коды р-на, для повторяющихся непустых геоточек
# Фильтрация и присвоение алиаса "a" для ctp_geo_dupl
ctp_geo_dupl_filtered = ctp_geo_dupl.filter(
    ctp_geo_dupl[ct.pickup_cl].isNotNull() & ctp_geo_dupl[ct.pickup_clon].isNotNull()
).alias("a")

# Присвоение алиаса "b" для ctp_geo
ctp_geo_alias = ctp_geo.alias("b")

# Выполнение join с использованием алиасов и квалифицированных имен столбцов
result = ctp_geo_dupl_filtered.join(
    ctp_geo_alias,
    (f.col("a." + ct.pickup_cl) == f.col("b." + ct.pickup_cl)) & 
    (f.col("a." + ct.pickup_clon) == f.col("b." + ct.pickup_clon))
).select(
    f.col("b." + ct.pickup_ct), 
    f.col("b." + ct.pickup_cl), 
    f.col("b." + ct.pickup_clon)
)

result.show()

Свяжем таблицы, используя SQL

In [ ]:
ctp_geo_dupl.createOrReplaceTempView("ctp_geo_dupl")
ctp_geo.createOrReplaceTempView("ctp_geo")

sql_query = f"""
SELECT b.`{ct.pickup_ct}`, b.`{ct.pickup_cl}`, b.`{ct.pickup_clon}`
FROM ctp_geo_dupl a
JOIN ctp_geo b
ON a.`{ct.pickup_cl}` = b.`{ct.pickup_cl}` 
AND a.`{ct.pickup_clon}` = b.`{ct.pickup_clon}`
WHERE a.`{ct.pickup_cl}` IS NOT NULL 
AND a.`{ct.pickup_clon}` IS NOT NULL
"""
result = spark.sql(sql_query)
result.show()

## df.selectExpr(...) и spark.sql(sql_query)

Инструкции `df.selectExpr(...)` и `spark.sql(sql_query)` выполняют схожие функции по выборке данных, но отличаются по способу использования и контексту:

### df.selectExpr(...)

`selectExpr` используется для выполнения выражений SQL на уровне DataFrame API. Это позволяет выполнять SQL-подобные выражения непосредственно на DataFrame без необходимости регистрации его как временной таблицы.
```python
df.selectExpr("col1", "col2", "col1 + col2 as sum_col").show()
```

- Преимущества:  
Прямое использование выражений SQL в контексте DataFrame.  
Не требует создания временных таблиц.  
Удобен для простых выборок и выражений.  

- Недостатки:  
Ограничен только выборкой данных и вычислением выражений в пределах одного DataFrame.  
Не подходит для выполнения сложных SQL-запросов с участием нескольких таблиц (например, join).  `

### spark.sql(sql_query)

`spark.sql(sql_query)` используется для выполнения полного SQL-запроса в контексте `SparkSession`. Это позволяет использовать весь синтаксис SQL, включая JOIN, GROUP BY, ORDER BY и другие сложные операции.

```python
sql_query = """
SELECT col1, col2, col1 + col2 as sum_col
FROM my_table
WHERE col1 > 10
"""
spark.sql(sql_query).shw()

-Преимущества:  
Полная поддержка SQL-синтаксиса.  
Подходит для сложных запросов, включающих несколько таблиц и сложные условия.  
Более знаком многим пользователям, имеющим опыт работы с SQL.  

- Недостатки:  
Требует регистрации DataFrame как временной таблицы с помощью createOrReplaceTempView.  
Может быть менее интуитивным для пользователей, предпочитающих работать с API DataFrame.  


```

## Метод .alias

Метод .alias в PySpark используется для двух основных целей:

### DataFrame - Присвоение псевдонимов 

Присвоение псевдонимов DataFrame позволяет удобно работать с именами таблиц, особенно при выполнении операций соединения (join), когда требуется различать столбцы из разных DataFrame.

In [ ]:
# Создание DataFrame
data1 = [("Alice", 10), ("Bob", 20)]
columns = ["name", "age"]
df1 = spark.createDataFrame(data1, columns)

data2 = [("Alice", "F"), ("Bob", "M")]
columns = ["name", "gender"]
df2 = spark.createDataFrame(data2, columns)

# Присвоение псевдонимов DataFrame
df1_alias = df1.alias("a")
df2_alias = df2.alias("b")

# Выполнение join с использованием псевдонимов
result = df1_alias.join(
    df2_alias, df1_alias["name"] == df2_alias["name"]
).select("a.name", "a.age", "b.gender")
result.show()

### Столбцы - Присвоение псевдонимов 

Присвоение псевдонимов столбцам позволяет переименовывать столбцы в результате выборки, что полезно для улучшения читаемости или для избежания конфликтов имен.

In [ ]:
# Создание DataFrame
data = [("Alice", 10), ("Bob", 20)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)

# Присвоение псевдонимов столбцам
df_with_alias = df.select(df["name"].alias("student_name"), df["age"].alias("student_age"))
df_with_alias.show()

## when

Функция when в PySpark используется для создания условных выражений в DataFrame. Она аналогична конструкции CASE WHEN в SQL и позволяет выполнять операции на основе определенных условий.

Синтаксис

In [ ]:
df.select(
    when(condition, value).otherwise(default_value)
)

**Простое условие**

In [ ]:
data = [
    ("Alice", 25),
    ("Bob", 22),
    ("Charlie", 30),
    ("Diana", 22),
    ("Eve", 28),
]

columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# Добавляем новый столбец "Age_25_or_Above"
df = df.withColumn("Age_25_or_Above", when(col("Age") >= 25, "Yes").otherwise("No"))
df.show()

```yaml
+-------+---+--------------+
|   Name|Age|Age_25_or_Above|
+-------+---+--------------+
|  Alice| 25|           Yes|
|    Bob| 22|            No|
|Charlie| 30|           Yes|
|  Diana| 22|            No|
|    Eve| 28|           Yes|
+-------+---+--------------+

**Несколько условий**

In [ ]:
df = df.withColumn(
    "Age_Category",
    when(col("Age") < 20, "Teenager")
    .when((col("Age") >= 20) & (col("Age") < 30), "Young Adult")
    .otherwise("Adult")
)
df.show()

```yaml
+-------+---+--------------+-------------+
|   Name|Age|Age_25_or_Above|Age_Category|
+-------+---+--------------+-------------+
|  Alice| 25|           Yes|        Adult|
|    Bob| 22|            No|  Young Adult|
|Charlie| 30|           Yes|        Adult|
|  Diana| 22|            No|  Young Adult|
|    Eve| 28|           Yes|  Young Adult|
+-------+---+--------------+-------------+

Вы можете цепочкой использовать несколько условий `when` и завершить цепочку `otherwise`, чтобы указать значение по умолчанию, если ни одно из условий не выполнено.

**Использование withColumn и when для обработки NULL значений**

In [ ]:
data = [
    ("Alice", 25, "Math"),
    ("Bob", None, "Physics"),
    ("Charlie", 30, None),
    ("Diana", 22, "History"),
    ("Eve", None, "Math"),
]

columns = ["Name", "Age", "Subject"]
df = spark.createDataFrame(data, columns)

# Заменяем NULL значения в столбце "Age" на 0
df = df.withColumn("Age", when(col("Age").isNull(), 0).otherwise(col("Age")))

# Заменяем NULL значения в столбце "Subject" на "Unknown"
df = df.withColumn("Subject", when(col("Subject").isNull(), "Unknown").otherwise(col("Subject")))

df.show()

```markdown
+-------+---+-------+
|   Name|Age|Subject|
+-------+---+-------+
|  Alice| 25|   Math|
|    Bob|  0|Physics|
|Charlie| 30|Unknown|
|  Diana| 22|History|
|    Eve|  0|   Math|
+-------+---+-------+

## .orderBy и .sort

Методы .orderBy и .sort в PySpark выполняют одинаковую функцию: они сортируют DataFrame по заданным столбцам. Эти методы являются синонимами и предоставляют одинаковые результаты. Они могут принимать как строковые имена столбцов, так и объекты Column. Оба метода также поддерживают сортировку в порядке возрастания и убывания.

Методы .orderBy и .sort предоставляют одинаковую функциональность, но существуют оба для удобства и читаемости кода. Разные пользователи могут предпочитать разные имена методов, но результат их применения будет одинаковым.

In [ ]:
# Пример данных
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
columns = ["Name", "Age"]

# Создание DataFrame
df = spark.createDataFrame(data, columns)

### orderBy

In [ ]:
# Сортировка по столбцу Age в порядке возрастания
df_ordered = df.orderBy("Age")
df_ordered.show()

# Сортировка по столбцу Age в порядке убывания
df_ordered_desc = df.orderBy(df["Age"].desc())
df_ordered_desc.show()

### sort

In [ ]:
# Сортировка по столбцу Age в порядке возрастания
df_sorted = df.sort("Age")
df_sorted.show()

# Сортировка по столбцу Age в порядке убывания
df_sorted_desc = df.sort(df["Age"].desc())
df_sorted_desc.show()

### Сортировка по нескольким столбцам

In [ ]:
# Сортировка по столбцу Age в порядке возрастания, затем по столбцу Name в порядке убывания
df_ordered_multi = df.orderBy(df["Age"].asc(), df["Name"].desc())
df_ordered_multi.show()

In [ ]:
# Сортировка по столбцу Age в порядке возрастания, затем по столбцу Name в порядке убывания
df_sorted_multi = df.sort(df["Age"].asc(), df["Name"].desc())
df_sorted_multi.show()

### Пример с DataFrame со структурными данными

In [ ]:
data = [
    ("Alice", ("New York", 10001)),
    ("Bob", ("San Francisco", 94105)),
    ("Cathy", ("Los Angeles", 90001))
]

schema = ["Name", "Address"]
address_schema = StructType([StructField("City", StringType(), True), StructField("ZIP", IntegerType(), True)])
df = spark.createDataFrame(data, schema=schema)

In [ ]:
# Сортировка по столбцу ZIP в порядке возрастания
df_sorted_by_zip = df.orderBy(col("Address.ZIP").asc())
df_sorted_by_zip.show(truncate=False

### asc_nulls_first, asc_nulls_last, desc_nulls_first, и desc_nulls_last

Методы asc_nulls_first, asc_nulls_last, desc_nulls_first, и desc_nulls_last используются для указания порядка сортировки с учетом null значений:

- asc_nulls_first: сортирует в порядке возрастания, помещая null значения в начало.
- asc_nulls_last: сортирует в порядке возрастания, помещая null значения в конец.
- desc_nulls_first: сортирует в порядке убывания, помещая null значения в начало.
- desc_nulls_last: сортирует в порядке убывания, помещая null значения в конец.

## Оконные функции

В `Apache Spark`, объект `Window` представляет собой спецификацию окна, которая определяет какие строки из DataFrame должны быть использованы для вычисления операций, таких как агрегации, ранжирования, или вычисления оконных функций. Оконные функции работают с группой строк (окно), которые определяются с помощью спецификации окна.

Основные компоненты спецификации окна включают:
- **PartitionBy**: определяет как разделить строки DataFrame на группы (партиции). Функции окна будут вычисляться независимо для каждой партиции.  
- **OrderBy**: определяет порядок сортировки строк внутри каждой партиции. Это необходимо для определения последовательности строк в окне.  
- **RowsBetween**, **RangeBetween**: определяют, какие строки в пределах каждой партиции будут включены в окно, используемое для вычислений.

### Вычисление суммы по группам с использованием окна

In [ ]:
 # Пример данных
data = [("Alice", 1), ("Bob", 3), ("Alice", 2), ("Bob", 5), ("Alice", 4)]

# Создание DataFrame
df = spark.createDataFrame(data, ["name", "score"])

# Определение спецификации окна
window_spec = Window.partitionBy("name").orderBy("score")

# Пример использования оконной функции (например, вычисление суммы score по каждому имени, упорядоченному по score)
df.withColumn("sum_score", f.sum("score").over(window_spec)).show()

```yaml
+-----+-----+---------+
| name|score|sum_score|
+-----+-----+---------+
|Alice|    1|        1|
|Alice|    2|        3|
|Alice|    4|        7|
|  Bob|    3|        3|
|  Bob|    5|        8|
+-----+-----+---------+

В этом примере `Window.partitionBy("name").orderBy("score")` создает спецификацию окна, которая разделяет строки DataFrame на партиции по столбцу "name" и упорядочивает строки в каждой партиции по столбцу "score". Функция `f.sum("score").over(window_spec)` применяет оконную функцию sum к столбцу "score" в рамках каждого окна, определенного спецификацией window_spec.

Аналог в SQL
```SQL
SELECT
    name,
    score,
    SUM(score) OVER (PARTITION BY name ORDER BY score) AS sum_score
FROM
    table_name;

### Вычисление ранга для каждого пользователя

In [ ]:
# Пример данных
data = [("Alice", 100), ("Bob", 80), ("Alice", 120), ("Bob", 90), ("Alice", 110)]

# Создание DataFrame
df = spark.createDataFrame(data, ["name", "score"])

# Определение спецификации окна для упорядочивания по убыванию score и разбиения по name
window_spec = Window.partitionBy("name").orderBy(df["score"].desc())

# Пример использования оконной функции для вычисления ранга (rank) в пределах каждого пользователя
df.withColumn("rank", f.rank().over(window_spec)).show()

```yaml
+-----+-----+----+
| name|score|rank|
+-----+-----+----+
|Alice|  120|   1|
|Alice|  110|   2|
|Alice|  100|   3|
|  Bob|   90|   1|
|  Bob|   80|   2|
+-----+-----+----+

Аналог в SQL
```SQL
SELECT
    name,
    score,
    RANK() OVER (PARTITION BY name ORDER BY score DESC) AS rank
FROM
    table_name;

### Порядковый номер для результата отбора

In [ ]:
# Assuming `ctp_geo_notnull_ct_null` and `ctd_geo_notnull_ct_null` are the DataFrames

# Union and assign sequential numbers in one step
combined_df = ctp_geo_notnull_ct_null.select(
    f.col(ct.pickup_ct).alias("census_tract"),
    f.col(ct.pickup_cl).alias("latitude"),
    f.col(ct.pickup_clon).alias("longitude")
).union(
    ctd_geo_notnull_ct_null.select(
        f.col(ct.dropoff_ct).alias("census_tract"),
        f.col(ct.dropoff_cl).alias("latitude"),
        f.col(ct.dropoff_clon).alias("longitude")
    )
).distinct().withColumn("census_tract", f.row_number().over(Window.orderBy(f.lit(1))))

## Типы данных

В PySpark используются следующие основные типы данных:

**Numeric Types (Числовые типы):**
- `ByteType`: Один байт, целое число от -128 до 127.
- `ShortType`: Два байта, целое число от -32,768 до 32,767.
- `IntegerType`: Четыре байта, целое число от -2,147,483,648 до 2,147,483,647.
- `LongType`: Восемь байт, целое число от -9,223,372,036,854,775,808 до 9,223,372,036,854,775,807.
- `FloatType`: Четыре байта, число с плавающей точкой одинарной точности.
- `DoubleType`: Восемь байт, число с плавающей точкой двойной точности.
- `DecimalType`: Десятичное число произвольной точности.

**String Types (Строковые типы):**
- `StringType`: Текстовая строка.

**Binary Types (Бинарные типы):**
- `BinaryType`: Массив байтов.

**Boolean Type (Булев тип):**
- `BooleanType`: Логическое значение (true или false).

**Datetime Types (Типы даты и времени):**
- `DateType`: Дата без времени.
- `TimestampType`: Дата и время с точностью до микросекунд.

**Complex Types (Комплексные типы):**
- `ArrayType`: Массив значений одного типа.
- `MapType`: Набор пар "ключ-значение", где ключи и значения могут быть разных типов.
- `StructType`: Структурированный тип, аналогичный строкам в таблице, содержащий несколько полей.

**Null Type (Тип Null):**
- `NullType`: Тип для значений NULL.


In [ ]:
# как получить тип конкретного поля.
df.schema["byte_col"].dataType

### Числовые типы

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ByteType, ShortType, IntegerType,
                              LongType, FloatType, DoubleType,
                              DecimalType, StructType, StructField

spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType([
    StructField("byte_col", ByteType(), True),
    StructField("short_col", ShortType(), True),
    StructField("int_col", IntegerType(), True),
    StructField("long_col", LongType(), True),
    StructField("float_col", FloatType(), True),
    StructField("double_col", DoubleType(), True),
    StructField("decimal_col", DecimalType(10, 2), True)
])

data = [
    (1, 2, 3, 4, 5.0, 6.0, 7.0)
]

df = spark.createDataFrame(data, schema)
df.show()

```yaml
+--------+---------+-------+-------+---------+----------+-----------+
|byte_col|short_col|int_col|long_col|float_col|double_col|decimal_col|
+--------+---------+-------+-------+---------+----------+-----------+
|       1|        2|      3|      4|      5.0|       6.0|        7.0|
+--------+---------+-------+-------+---------+----------+-----------+
```как получить тип конкретного поля.

### Типы даты и времени

#### Пример полей даты-времени

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType, StructType, StructField

spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType([
    StructField("date_col", DateType(), True),
    StructField("timestamp_col", TimestampType(), True)
])

data = [
    ("2024-07-07", "2024-07-07 12:34:56")
]

df = spark.createDataFrame(data, schema)
df.show()

```yaml
+----------+-------------------+
|  date_col|      timestamp_col|
+----------+-------------------+
|2024-07-07|2024-07-07 12:34:56|
+----------+-------------------+
```

#### Основные функции для дат

Вот несколько основных функций PySpark для работы с датами и временем:

In [ ]:
from pyspark.sql import functions as f

# 1. Добавление или вычитание дней
df = df.withColumn("date_add", f.date_add("date_col", 10))
df = df.withColumn("date_sub", f.date_sub("date_col", 10))

# 2. Вычисление разницы между датами
df = df.withColumn("datediff", f.datediff(f.to_date(f.lit("2024-08-01")), "date_col"))

# 3. День недели (1 - понедельник, 7 - воскресенье)
df = df.withColumn("dayofweek", f.dayofweek("date_col"))

# 4. Извлечение года, месяца, дня и т.д.
df = df.withColumn("year", f.year("date_col"))
df = df.withColumn("month", f.month("date_col"))
df = df.withColumn("day", f.dayofmonth("date_col"))

# 5. Форматирование даты в строку
df = df.withColumn("date_format", f.date_format("date_col", "yyyy-MM-dd"))

# 6. Преобразование строки в дату (если формат известен)
df = df.withColumn("to_date", f.to_date("date_string_col", "yyyy-MM-dd"))

# 7. Получение текущего времени
df = df.withColumn("current_timestamp", f.current_timestamp())

# 8. Перевод времени из одной временной зоны в другую
df = df.withColumn("from_utc_timestamp", f.from_utc_timestamp("timestamp_col", "GMT"))

# 9. Проверка на високосный год
df = df.withColumn("is_leap_year", f.year("date_col").cast("int").rdd.map(lambda x: x[0]).map(lambda x: (x % 4 == 0 and x % 100 != 0) or (x % 400 == 0)))

# 10. Поиск первой даты после текущей
df = df.withColumn("next_day", f.next_day(f.current_date(), "monday"))

# 11. Подсчет количества дней между двумя датами
df = df.withColumn("datediff", f.datediff(f.to_date(f.lit("2024-08-01")), "date_col"))


#### Выделение части времени

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType, StructType, StructField
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType([
    StructField("date_col", DateType(), True),
    StructField("timestamp_col", TimestampType(), True)
])

data = [
    ("2024-07-07", "2024-07-07 12:34:56")
]

df = spark.createDataFrame(data, schema)

df = df.withColumn("year", f.year("timestamp_col"))
df = df.withColumn("month", f.month("timestamp_col"))
df = df.withColumn("day", f.dayofmonth("timestamp_col"))
df = df.withColumn("hour", f.hour("timestamp_col"))
df = df.withColumn("minute", f.minute("timestamp_col"))
df = df.withColumn("second", f.second("timestamp_col"))

df.show()

```yaml
+----------+-------------------+----+-----+---+----+------+------+
|  date_col|      timestamp_col|year|month|day|hour|minute|second|
+----------+-------------------+----+-----+---+----+------+------+
|2024-07-07|2024-07-07 12:34:56|2024|    7|  7|  12|    34|    56|
+----------+-------------------+----+-----+---+----+------+------+
```

#### Сдвиг add sub

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType, StructType, StructField
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType([
    StructField("date_col", DateType(), True),
    StructField("timestamp_col", TimestampType(), True)
])

data = [
    ("2024-07-07", "2024-07-07 12:34:56")
]

df = spark.createDataFrame(data, schema)

df = df.withColumn("date_add", f.date_add("timestamp_col", 10))
df = df.withColumn("date_sub", f.date_sub("timestamp_col", 10))

df.show()

```yaml
+----------+-------------------+-------------------+-------------------+
|  date_col|      timestamp_col|           date_add|           date_sub|
+----------+-------------------+-------------------+-------------------+
|2024-07-07|2024-07-07 12:34:56|2024-07-17 12:34:56|2024-06-27 12:34:56|
+----------+-------------------+-------------------+-------------------+
```

#### Разница datediff

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType, StructType, StructField
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType([
    StructField("date_col", DateType(), True),
    StructField("timestamp_col", TimestampType(), True)
])

data = [
    ("2024-07-07", "2024-07-07 12:34:56")
]

df = spark.createDataFrame(data, schema)

df = df.withColumn("datediff_current_date", f.datediff(f.current_date(), "date_col"))
df = df.withColumn("datediff_date_col", f.datediff(f.to_date(f.lit("2024-08-01")), "date_col"))

df.show()

```yaml
+----------+-------------------+---------------------+------------------+
|  date_col|      timestamp_col|datediff_current_date|datediff_date_col |
+----------+-------------------+---------------------+------------------+
|2024-07-07|2024-07-07 12:34:56|                  361|                25|
+----------+-------------------+---------------------+------------------+
```

#### Символы формата

В PySpark можно использовать следующие символы формата для аргумента format в функции f.to_timestamp:

**Год:**
- **y или yy:** 2-значный год (например, 21 для 2021).
- **yyyy:** 4-значный год (например, 2021).

**Месяц:**
- **M или MM:** Месяц как число без ведущего нуля (1-12).
- **MMM или MMMM:** Название месяца (янв, январь и т.д.).

**День:**
- **d или dd:** День месяца как число без ведущего нуля (1-31).

**Час:**
- **H или HH:** Часы в 24-часовом формате без ведущего нуля (0-23).
- **h или hh:** Часы в 12-часовом формате с ведущим нулем (01-12).

**Минуты:**
- **m или mm:** Минуты без ведущего нуля (0-59).

**Секунды:**
- **s или ss:** Секунды без ведущего нуля (0-59).

**Фракции секунд:**
- **S или SSS:** Фракции секунды.

**AM/PM:**
- **a:** AM или PM.

**Примеры:**

- "**yyyy-MM-dd HH:mm:ss:** Дата и время в формате год-месяц-день час:минута:секунда."
- "**yyyy-MM-dd:** Только дата в формате год-месяц-день."
- "**MM/dd/yyyy:** Дата в формате месяц/день/год."
- "**MMM dd, yyyy HH:mm:ss:** Дата и время с использованием аббревиатуры месяца, например, янв 01, 2021 12:34:56."

#### Форматирование даты

Для форматирования дат в PySpark основными функциями являются `f.date_format` для преобразования даты в строку и `f.to_timestamp` для преобразования строки в формат даты/времени.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType, StructType, StructField
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType([
    StructField("date_col", DateType(), True)
])

data = [
    ("2024-07-07"),
    ("2024-08-15"),
    ("2023-05-10")
]

df = spark.createDataFrame(data, schema)

# Форматирование даты в строку
df = df.withColumn("date_format", f.date_format("date_col", "yyyy-MM-dd"))

# Преобразование строки в Timestamp
df = df.withColumn("to_timestamp", f.to_timestamp("date_col"))

# Другие возможные сценарии форматирования даты
df = df.withColumn("month_day", f.date_format("date_col", "MM-dd"))
df = df.withColumn("week_day", f.date_format("date_col", "E"))

df.show()

```yaml
+----------+------------+-------------------+----------+--------+
|  date_col| date_format|      to_timestamp| month_day|week_day|
+----------+------------+-------------------+----------+--------+
|2024-07-07|  2024-07-07|2024-07-07 00:00:00|     07-07|     Sat|
|2024-08-15|  2024-08-15|2024-08-15 00:00:00|     08-15|     Thu|
|2023-05-10|  2023-05-10|2023-05-10 00:00:00|     05-10|     Wed|
+----------+------------+-------------------+----------+--------+
```

##### f.to_timestamp

Функция f.to_timestamp(col, format=None) в PySpark используется для преобразования столбца или выражения колонки в тип данных TimestampType.  

Если формат не указан, PySpark будет использовать формат по умолчанию, что может привести к некорректным результатам, если формат данных отличается от ожидаемого.  
При использовании функции to_timestamp необходимо учитывать, что строки, которые не могут быть интерпретированы как даты в указанном формате, будут приводить к значению NULL в соответствующем столбце.

**Аргументы:**
- col: Колонка или выражение колонки, которое необходимо преобразовать в тип TimestampType.
- format (опционально): Строка, определяющая формат входной строки. Если не указан, PySpark попытается автоматически определить формат.

In [ ]:
# Создаем DataFrame с колонкой date_col типа StringType
data = [("2024-07-07",), ("2024-08-15",), ("2023-05-10",)]
df = spark.createDataFrame(data, ["date_col"])

# Преобразуем колонку date_col в тип TimestampType
df = df.withColumn("to_timestamp", f.to_timestamp("date_col"))
df = df.withColumn("to_timestamp_custom", f.to_timestamp("date_col", "yyyy-MM-dd"))

df.show()

```yaml
+----------+-------------------+-------------------+
|  date_col|      to_timestamp |to_timestamp_custom|
+----------+-------------------+-------------------+
|2024-07-07|2024-07-07 00:00:00|2024-07-07 00:00:00|
|2024-08-15|2024-08-15 00:00:00|2024-08-15 00:00:00|
|2023-05-10|2023-05-10 00:00:00|2023-05-10 00:00:00|
+----------+-------------------+-------------------+
```

##### f.date_format

Функция `f.date_format(col, format)` в PySpark используется для форматирования столбца даты или времени в строку с заданным форматом.  

Функция `date_format` позволяет указать любой формат, совместимый с `Java SimpleDateFormat`, что обеспечивает гибкость в форматировании дат и времени в PySpark.
При использовании формата следует учитывать, что некорректный формат может привести к ошибкам или непредсказуемым результатам.

**Аргументы:**

- col: Колонка или выражение колонки, содержащее дату или время, которое необходимо отформатировать.
- format: Строка, определяющая формат вывода. Формат должен соответствовать спецификациям Java SimpleDateFormat.

In [ ]:
# Создаем DataFrame с колонкой date_col типа DateType
data = [("2024-07-07",), ("2024-08-15",), ("2023-05-10",)]
df = spark.createDataFrame(data, ["date_col"]).withColumn("date_col", f.col("date_col").cast(DateType()))

# Преобразуем колонку date_col в строку с заданным форматом
df = df.withColumn("formatted_date", f.date_format("date_col", "yyyy-MM-dd"))
df.show()

```yaml
+----------+---------------+
|  date_col| formatted_date|
+----------+---------------+
|2024-07-07|     2024-07-07|
|2024-08-15|     2024-08-15|
|2023-05-10|     2023-05-10|
+----------+---------------+


In [ ]:
df = df.withColumn("formatted_date", f.date_format("date_col", "MM/dd/yyyy"))

df.show()

```yaml
+----------+---------------+
|  date_col| formatted_date|
+----------+---------------+
|2024-07-07|     07/07/2024|
|2024-08-15|     08/15/2024|
|2023-05-10|     05/10/2023|
+----------+---------------+
```

## Сохранение в файл

In [ ]:
# сохранить DataFrame taxi в формате CSV в директории /work/data
taxi.write.csv("/work/data/taxi.csv", header=True)

In [ ]:
# перезапись
taxi.write.csv("/work/data/taxi.csv", header=True, mode="overwrite")

Когда вы используете метод `write.csv()` в Spark, он создает несколько файлов в формате CSV, так как каждый раздел (partition) DataFrame сохраняется в отдельный файл.  
Используйте `coalesce(1)` для сохранения в единый файл CSV

In [ ]:
# Сохранение DataFrame в единый файл CSV
taxi.coalesce(1).write.csv("/work/data/taxi.csv", header=True)

Есть несколько важных преимуществ и различий между сохранением DataFrame в CSV с сохранением каждого раздела в отдельный файл и с использованием метода `coalesce(1)` для сохранения в один файл:

**Параллелизм и производительность:**
- Сохранение каждого раздела в отдельный файл CSV в Spark делает процесс сохранения параллельным. Каждый раздел обрабатывается и сохраняется независимо, что может улучшить производительность на больших объемах данных.
- Использование `coalesce(1)` приводит к тому, что все данные будут перемещены и записаны на одном узле, что может быть менее эффективно для больших наборов данных из-за ограничений ресурсов одного узла.

**Управление файлами:**
- При сохранении каждого раздела в отдельный файл у вас будет много файлов, что может быть неудобно для управления и обработки в дальнейшем.
- С использованием `coalesce(1)` вы получите только один файл, что облегчает управление результатами и последующую обработку.

**Использование ресурсов:**
- Сохранение каждого раздела в отдельный файл может быть более эффективным с точки зрения распределения нагрузки на узлы кластера Spark и использования ресурсов.
- Однако использование `coalesce(1)` может быть полезным в случаях, когда вам действительно нужен единый файл и вы готовы потерпеть некоторое ухудшение производительности из-за работы на одном узле.

Cоздается папка `taxis.csv` со следующими файлами:  
`._SUCCESS.crc`     1кб  
Это скрытый файл, который указывает на успешное завершение процесса сохранения данных. Он содержит информацию о том, что операция сохранения была выполнена без ошибок.  

`.part-00000-45c80058-7aeb-4e94-82a8-570964c6aceb-c000.csv.crc`       20.3мб  
Этот файл .crc является контрольной суммой (CRC) для соответствующего CSV файла part-00000-45c80058-7aeb-4e94-82a8-570964c6aceb-c000.csv. Он используется для проверки целостности данных в CSV файле.

`_SUCCESS`        0кб  
Этот файл пустой и также служит индикатором успешного завершения операции сохранения. Он может быть создан в случае использования определенных форматов записи в Spark.

`part-00000-45c80058-7aeb-4e94-82a8-570964c6aceb-c000.csv`        2.5гб  
Это основной CSV файл, который содержит данные вашего DataFrame. В данном случае, это единственный файл из-за использования `coalesce(1)`, который объединяет все разделы DataFrame в один файл.

Дополнительные файлы (.crc и _SUCCESS) важны для контроля целостности данных и индикации успешного выполнения операции. Они могут быть полезны при автоматизации и мониторинге процессов обработки данных в Spark.  

Можно перенести и использовать только основной CSV файл `part-00000-45c80058-7aeb-4e94-82a8-570964c6aceb-c000.csv`, который содержит фактические данные DataFrame. Остальные файлы, такие как `.crc` файлы и `_SUCCESS`, являются вспомогательными и могут быть удалены или не использованы при переносе данных на другой диск.
